# Lesson 1 // Data wrangling 

---
In this lesson we'll:

1. see how to save .csv data in the form of an .RData object
2. introduce data transformations using the **dplyr** package, using the five key dplyr verbs: 

    + filter
    + arrange
    + mutate
    + summarise
    + grouped_by


3. introduce the pipe operator `%>%`
4. see a few nice things you can do by combining dplyr verbs (grouped filters and mutates, for example)
5. use the dplyr verbs to build a small movie rating dataset that we'll use in the next lesson on recommender systems.
6. introduce various *join* operations that can be used to combine information across multiple tables (relational data)

### Sources and references

 * http://r4ds.had.co.nz/transform.html
 * http://r4ds.had.co.nz/relational-data.html
 ---

## Get the MovieLens data and save as .RData

[MovieLens](https://grouplens.org/datasets/movielens/) is a great resource for data on movie ratings. The full dataset has ratings on 40 000 movies by 260 000 users, some 24 million ratings in all. We'll use a smaller dataset with ratings of 9 000 movies by 700 users (100 000 ratings in all). 

Download the file "ml-latest-small.zip" from https://grouplens.org/datasets/movielens/ and unzip it to the *data* directory in your main project folder (make a folder called *data* if you haven't already). You should see four csv files: `links.csv`, `movies.csv`, `ratings.csv`, and `tags.csv`.


First let's save the data we downloaded as an .RData object. .RData objects are smaller than csv, plus we can save all four csvs in a single .RData object that we can call with a single call to `load` the dataset later on.

In [1]:
# read in the csv files
links <- read.csv("data/ml-latest-small/links.csv")
movies <- read.csv("data/ml-latest-small/movies.csv")
ratings <- read.csv("data/ml-latest-small/ratings.csv")
tags <- read.csv("data/ml-latest-small/tags.csv")

# save as .RData
save(links,movies,ratings,tags,file="data/movielens-small.RData")

# check that its worked
rm(list=ls())
load("data/movielens-small.RData")

You'll only need to do the above part once so, once you've got the data saved as .RData, start running the notebook from here.

## Loading the tidyverse

Load the **tidyverse** collection of packages, which loads the following packages: **ggplot2**, **tibble**, **tidyr**, **readr**, **purrr**, and **dplyr**.

In [2]:
library(tidyverse)

Loading tidyverse: ggplot2
Loading tidyverse: tibble
Loading tidyverse: tidyr
Loading tidyverse: readr
Loading tidyverse: purrr
Loading tidyverse: dplyr
Conflicts with tidy packages ---------------------------------------------------
filter(): dplyr, stats
lag():    dplyr, stats


Load the MovieLens data

In [3]:
load("data/movielens-small.RData")

Tibbles are a special kind of dataframe that work well with tidyverse packages ("in the tidyverse" in tidyversese). 

In [4]:
# convert ratings to a "tibble"
ratings <- as.tibble(ratings)

A nice feature of tibbles is that if you display them in the console (by typing `ratings`, for example) only first few rows and columns are shown. Unfortunately this doesn't carry over to jupyter notebook, so I need to explicitly say `print(ratings)` or all the rows are shown.

In [5]:
print(ratings)

# A tibble: 100,004 x 4
   userId movieId rating  timestamp
    <int>   <int>  <dbl>      <int>
 1      1      31    2.5 1260759144
 2      1    1029    3.0 1260759179
 3      1    1061    3.0 1260759182
 4      1    1129    2.0 1260759185
 5      1    1172    4.0 1260759205
 6      1    1263    2.0 1260759151
 7      1    1287    2.0 1260759187
 8      1    1293    2.0 1260759148
 9      1    1339    3.5 1260759125
10      1    1343    2.0 1260759131
# ... with 99,994 more rows


Explore some of the variables

In [6]:
str(ratings)

Classes 'tbl_df', 'tbl' and 'data.frame':	100004 obs. of  4 variables:
 $ userId   : int  1 1 1 1 1 1 1 1 1 1 ...
 $ movieId  : int  31 1029 1061 1129 1172 1263 1287 1293 1339 1343 ...
 $ rating   : num  2.5 3 3 2 4 2 2 2 3.5 2 ...
 $ timestamp: int  1260759144 1260759179 1260759182 1260759185 1260759205 1260759151 1260759187 1260759148 1260759125 1260759131 ...


In [7]:
glimpse(ratings)

Observations: 100,004
Variables: 4
$ userId    <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
$ movieId   <int> 31, 1029, 1061, 1129, 1172, 1263, 1287, 1293, 1339, 1343,...
$ rating    <dbl> 2.5, 3.0, 3.0, 2.0, 4.0, 2.0, 2.0, 2.0, 3.5, 2.0, 2.5, 1....
$ timestamp <int> 1260759144, 1260759179, 1260759182, 1260759185, 126075920...


In [8]:
glimpse(movies)

Observations: 9,125
Variables: 3
$ movieId <int> 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, ...
$ title   <fctr> Toy Story (1995), Jumanji (1995), Grumpier Old Men (1995),...
$ genres  <fctr> Adventure|Animation|Children|Comedy|Fantasy, Adventure|Chi...


We'll look at database joins in more detail, but for now, this just adds movie title to the `ratings` data by pulling that information from `movies`.

In [9]:
ratings <- left_join(ratings, movies)

Joining, by = "movieId"


## Filtering rows with `filter()`

Here we illustrate the use of `filter()` by extracting user 1's observations from the *ratings* data frame.

In [10]:
u1 <- filter(ratings, userId == 1)
u1

userId,movieId,rating,timestamp,title,genres
1,31,2.5,1260759144,Dangerous Minds (1995),Drama
1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical
1,1061,3.0,1260759182,Sleepers (1996),Thriller
1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller
1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama
1,1263,2.0,1260759151,"Deer Hunter, The (1978)",Drama|War
1,1287,2.0,1260759187,Ben-Hur (1959),Action|Adventure|Drama
1,1293,2.0,1260759148,Gandhi (1982),Drama
1,1339,3.5,1260759125,Dracula (Bram Stoker's Dracula) (1992),Fantasy|Horror|Romance|Thriller
1,1343,2.0,1260759131,Cape Fear (1991),Thriller


Next we extract the observations for user 1 that received a rating greater than 3. Multiple filter conditions are created with `&` (and) and `|` (or).

In [11]:
filter(ratings, userId == 1 & rating > 3)

userId,movieId,rating,timestamp,title,genres
1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama
1,1339,3.5,1260759125,Dracula (Bram Stoker's Dracula) (1992),Fantasy|Horror|Romance|Thriller
1,1953,4.0,1260759191,"French Connection, The (1971)",Action|Crime|Thriller
1,2105,4.0,1260759139,Tron (1982),Action|Adventure|Sci-Fi


Here's another way of writing the same condition as above:

In [12]:
filter(ratings, userId == 1, rating > 3)

userId,movieId,rating,timestamp,title,genres
1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama
1,1339,3.5,1260759125,Dracula (Bram Stoker's Dracula) (1992),Fantasy|Horror|Romance|Thriller
1,1953,4.0,1260759191,"French Connection, The (1971)",Action|Crime|Thriller
1,2105,4.0,1260759139,Tron (1982),Action|Adventure|Sci-Fi


The `%in%` command is often useful when using dplyr verbs:

In [13]:
filter(ratings, userId == 1, rating %in% c(1,4))

userId,movieId,rating,timestamp,title,genres
1,1172,4,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama
1,1405,1,1260759203,Beavis and Butt-Head Do America (1996),Adventure|Animation|Comedy|Crime
1,1953,4,1260759191,"French Connection, The (1971)",Action|Crime|Thriller
1,2105,4,1260759139,Tron (1982),Action|Adventure|Sci-Fi
1,2968,1,1260759200,Time Bandits (1981),Adventure|Comedy|Fantasy|Sci-Fi


## Introducing the pipe

The pipe operator `%>%` is a very useful way of chaining together multiple operations. A typical format is something like:

*data* `%>%` *operation 1* `%>%` *operation 2* 

You read the code from left to right: Start with *data*, apply some operation (operation 1) to it, get a result, and then apply another operation (operation 2) to that result, to generate another result (the final result, in this example). A useful way to think of the pipe is as similar to "then".

The main goal of the pipe is to make code easier, by focusing on the transformations rather than on what is being transformed. Usually this is the case, but its also possible to get carried away and end up with a huge whack of piped statements. Deciding when to break a block up is an art best learned by experience. 

In [14]:
# filtering with the pipe
ratings %>% filter(userId == 1)

userId,movieId,rating,timestamp,title,genres
1,31,2.5,1260759144,Dangerous Minds (1995),Drama
1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical
1,1061,3.0,1260759182,Sleepers (1996),Thriller
1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller
1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama
1,1263,2.0,1260759151,"Deer Hunter, The (1978)",Drama|War
1,1287,2.0,1260759187,Ben-Hur (1959),Action|Adventure|Drama
1,1293,2.0,1260759148,Gandhi (1982),Drama
1,1339,3.5,1260759125,Dracula (Bram Stoker's Dracula) (1992),Fantasy|Horror|Romance|Thriller
1,1343,2.0,1260759131,Cape Fear (1991),Thriller


The main usefulness of the pipe is when combining multiple operations

In [15]:
# first filter on userId then on rating
u1_likes <- ratings %>% filter(userId == 1) %>% filter(rating > 3)
u1_likes

userId,movieId,rating,timestamp,title,genres
1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama
1,1339,3.5,1260759125,Dracula (Bram Stoker's Dracula) (1992),Fantasy|Horror|Romance|Thriller
1,1953,4.0,1260759191,"French Connection, The (1971)",Action|Crime|Thriller
1,2105,4.0,1260759139,Tron (1982),Action|Adventure|Sci-Fi


In [16]:
# another way of doing the same thing
ratings %>% filter(userId == 1 & rating > 3)

userId,movieId,rating,timestamp,title,genres
1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama
1,1339,3.5,1260759125,Dracula (Bram Stoker's Dracula) (1992),Fantasy|Horror|Romance|Thriller
1,1953,4.0,1260759191,"French Connection, The (1971)",Action|Crime|Thriller
1,2105,4.0,1260759139,Tron (1982),Action|Adventure|Sci-Fi


## Arranging rows with `arrange()`

Ordering user 1's "liked" movies in descending order of rating (note the use of `desc`)

In [17]:
arrange(u1_likes, desc(rating))

userId,movieId,rating,timestamp,title,genres
1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama
1,1953,4.0,1260759191,"French Connection, The (1971)",Action|Crime|Thriller
1,2105,4.0,1260759139,Tron (1982),Action|Adventure|Sci-Fi
1,1339,3.5,1260759125,Dracula (Bram Stoker's Dracula) (1992),Fantasy|Horror|Romance|Thriller


Subsequent arguments to `arrange()` can be used to arrange by multiple columns. Here we first order user 1's liked movies by rating (in descending order) and then by timestamp (in ascending order)

In [18]:
arrange(u1_likes, desc(rating),timestamp)

userId,movieId,rating,timestamp,title,genres
1,2105,4.0,1260759139,Tron (1982),Action|Adventure|Sci-Fi
1,1953,4.0,1260759191,"French Connection, The (1971)",Action|Crime|Thriller
1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama
1,1339,3.5,1260759125,Dracula (Bram Stoker's Dracula) (1992),Fantasy|Horror|Romance|Thriller


We can also use the pipe to do the same thing

In [19]:
u1_likes %>% arrange(desc(rating))

userId,movieId,rating,timestamp,title,genres
1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama
1,1953,4.0,1260759191,"French Connection, The (1971)",Action|Crime|Thriller
1,2105,4.0,1260759139,Tron (1982),Action|Adventure|Sci-Fi
1,1339,3.5,1260759125,Dracula (Bram Stoker's Dracula) (1992),Fantasy|Horror|Romance|Thriller


Finally, here's an example of combining filter and arrange operations with the pipe

In [20]:
ratings %>% filter(userId == 1 & rating > 3) %>% arrange(desc(rating))

userId,movieId,rating,timestamp,title,genres
1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama
1,1953,4.0,1260759191,"French Connection, The (1971)",Action|Crime|Thriller
1,2105,4.0,1260759139,Tron (1982),Action|Adventure|Sci-Fi
1,1339,3.5,1260759125,Dracula (Bram Stoker's Dracula) (1992),Fantasy|Horror|Romance|Thriller


## Selecting columns with `select()`

Select is a bit like `filter()` for columns. The syntax is straightforward, the first argument gives the dataframe, and then you list the variables you want to select!

In [21]:
select(u1_likes,title,rating)

title,rating
Cinema Paradiso (Nuovo cinema Paradiso) (1989),4.0
Dracula (Bram Stoker's Dracula) (1992),3.5
"French Connection, The (1971)",4.0
Tron (1982),4.0


To exclude variables just put a minus sign in front of them

In [22]:
select(u1_likes,-userId,-timestamp)

movieId,rating,title,genres
1172,4.0,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama
1339,3.5,Dracula (Bram Stoker's Dracula) (1992),Fantasy|Horror|Romance|Thriller
1953,4.0,"French Connection, The (1971)",Action|Crime|Thriller
2105,4.0,Tron (1982),Action|Adventure|Sci-Fi


You can also use `select()` to reorder variables. A useful helpful function here is `everything()`.

In [23]:
# original order
u1_likes

# reorder so title is first
select(u1_likes, title, everything())

userId,movieId,rating,timestamp,title,genres
1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama
1,1339,3.5,1260759125,Dracula (Bram Stoker's Dracula) (1992),Fantasy|Horror|Romance|Thriller
1,1953,4.0,1260759191,"French Connection, The (1971)",Action|Crime|Thriller
1,2105,4.0,1260759139,Tron (1982),Action|Adventure|Sci-Fi


title,userId,movieId,rating,timestamp,genres
Cinema Paradiso (Nuovo cinema Paradiso) (1989),1,1172,4.0,1260759205,Drama
Dracula (Bram Stoker's Dracula) (1992),1,1339,3.5,1260759125,Fantasy|Horror|Romance|Thriller
"French Connection, The (1971)",1,1953,4.0,1260759191,Action|Crime|Thriller
Tron (1982),1,2105,4.0,1260759139,Action|Adventure|Sci-Fi


## Adding new variables with `mutate()`

Mutating operations add a new column to a dataframe. Here's a trivial example to get started:

In [24]:
mutate(u1_likes, this_is = "stupid")  

userId,movieId,rating,timestamp,title,genres,this_is
1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama,stupid
1,1339,3.5,1260759125,Dracula (Bram Stoker's Dracula) (1992),Fantasy|Horror|Romance|Thriller,stupid
1,1953,4.0,1260759191,"French Connection, The (1971)",Action|Crime|Thriller,stupid
1,2105,4.0,1260759139,Tron (1982),Action|Adventure|Sci-Fi,stupid


A more useful use of mutate is to construct new variable based on existing variables. This is the way that `mutate` is almost always used.

In [25]:
mutate(u1, like = ifelse(rating > 3, 1, 0))  

userId,movieId,rating,timestamp,title,genres,like
1,31,2.5,1260759144,Dangerous Minds (1995),Drama,0
1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical,0
1,1061,3.0,1260759182,Sleepers (1996),Thriller,0
1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller,0
1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama,1
1,1263,2.0,1260759151,"Deer Hunter, The (1978)",Drama|War,0
1,1287,2.0,1260759187,Ben-Hur (1959),Action|Adventure|Drama,0
1,1293,2.0,1260759148,Gandhi (1982),Drama,0
1,1339,3.5,1260759125,Dracula (Bram Stoker's Dracula) (1992),Fantasy|Horror|Romance|Thriller,1
1,1343,2.0,1260759131,Cape Fear (1991),Thriller,0


We can also use the pipe for mutating operations. Hopefully you're getting used to the pipe by now, so let's embed a mutating operation within a larger pipe than we've used before. 

In [26]:
ratings %>% 
mutate(like = ifelse(rating > 3, 1, 0)) %>% 
filter(userId == 1) %>% 
select(like, everything()) 

like,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama
0,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical
0,1,1061,3.0,1260759182,Sleepers (1996),Thriller
0,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller
1,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama
0,1,1263,2.0,1260759151,"Deer Hunter, The (1978)",Drama|War
0,1,1287,2.0,1260759187,Ben-Hur (1959),Action|Adventure|Drama
0,1,1293,2.0,1260759148,Gandhi (1982),Drama
1,1,1339,3.5,1260759125,Dracula (Bram Stoker's Dracula) (1992),Fantasy|Horror|Romance|Thriller
0,1,1343,2.0,1260759131,Cape Fear (1991),Thriller


## Aggregating over rows with `summarise()`

The `summarise()` verb (or `summarize()` will also work) summarises the rows in a data frame in some way. When applied to the whole data frame, it will collapse it to a single row. For example, here we take user 1's data, and calculate their average rating and the number of movies they have given a rating higher than 3 to:

In [27]:
summarise(u1, mean = mean(rating), likes = sum(rating > 3))

mean,likes
2.55,4


You need to watch out for NAs when using `summarise()`. If one exists, operations like `mean()` will return NA. You can exclude NAs from calculations using `na.rm = TRUE`:

In [28]:
# introduce an NA
u1$rating[1] <- NA

# see what happens
summarise(u1, mean = mean(rating), likes = sum(rating > 3))

mean,likes
NA,NA


In [29]:
# with na.rm = T
summarise(u1, mean = mean(rating, na.rm = T), likes = sum(rating > 3, na.rm = T))

mean,likes
2.552632,4


`summarise()` is most useful when combined with `group_by()`, which imposes a grouping structure on a data frame. After applying `group_by()`, subsequent dplyr verbs will be applied to individual groups, basically repeating the code for each group. That means that `summarise()` will calculate a summary for each group:

In [30]:
# tell dplyr to group ratings by userId
ratings_by_user <- group_by(ratings, userId)

# apply summarize() to see how many movies each user has rated
ratings_by_user %>% summarize(count = n()) %>% head()

userId,count
1,20
2,76
3,51
4,204
5,100
6,44


In [31]:
# get sorted counts (plus some presentation stuff)
ratings %>% 
group_by(userId) %>% 
summarize(count = n()) %>% 
arrange(desc(count)) %>% 
head(20) %>%     # take first two rows
t()  # transpose 

userId,547,564,624,15,73,452,468,380,311,30,294,509,580,213,212,472,388,23,457,518
count,2391,1868,1735,1700,1610,1340,1291,1063,1019,1011,947,923,922,910,876,830,792,726,713,707


In [32]:
# or with the pipe (last time)
ratings %>% group_by(userId) %>% summarize(count = n()) %>% head(10)

userId,count
1,20
2,76
3,51
4,204
5,100
6,44
7,88
8,116
9,45
10,46


## Other uses of `grouped_by()`: grouped filters and grouped mutates
While you'll probably use `group_by()` most often with `summarise()`, it can also be useful when used in conjunction with `filter()` and `mutate()`. Grouped filters perform the filtering within each group. Below we use it to extract each user's favourite movie (or movies, if there's a tie).

In [33]:
# example of a grouped filter
ratings %>% group_by(userId) %>% filter(rank(desc(rating)) < 2)

userId,movieId,rating,timestamp,title,genres
13,1,5.0,1331380058,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
13,356,5.0,1331380018,Forrest Gump (1994),Comedy|Drama|Romance|War
14,3175,5.0,976244313,Galaxy Quest (1999),Adventure|Comedy|Sci-Fi
18,25,5.0,856006886,Leaving Las Vegas (1995),Drama|Romance
18,32,5.0,856006885,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
24,6,5.0,849321588,Heat (1995),Action|Crime|Thriller
24,296,5.0,849282414,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
35,3072,5.0,1174450056,Moonstruck (1987),Comedy|Romance
50,589,5.0,847412628,Terminator 2: Judgment Day (1991),Action|Sci-Fi
72,55820,5.0,1464722872,No Country for Old Men (2007),Crime|Drama


Here we use a grouped mutate to standardise each user's ratings so that they have a mean of zero (for each user, which guarantees the overall mean rating is also zero).

In [34]:
# example of a grouped mutate
ratings %>% 
group_by(userId) %>%
mutate(centered_rating = rating - mean(rating)) %>% 
select(-movieId,-timestamp,-genres)

userId,rating,title,centered_rating
1,2.5,Dangerous Minds (1995),-0.0500000
1,3.0,Dumbo (1941),0.4500000
1,3.0,Sleepers (1996),0.4500000
1,2.0,Escape from New York (1981),-0.5500000
1,4.0,Cinema Paradiso (Nuovo cinema Paradiso) (1989),1.4500000
1,2.0,"Deer Hunter, The (1978)",-0.5500000
1,2.0,Ben-Hur (1959),-0.5500000
1,2.0,Gandhi (1982),-0.5500000
1,3.5,Dracula (Bram Stoker's Dracula) (1992),0.9500000
1,2.0,Cape Fear (1991),-0.5500000


## Putting it all together: extracting a sample set of reviews for Lesson 2

In this section we'll take what we've learned and do something useful: build a 15x20 matrix containing the reviews made on 20 movies by 15 users. We'll use this matrix in the next lesson to build a recommendation system.

First, we select the 15 users we want to use. I've chosen to use 15 users with moderately frequent viewing habits (remember there are 700 users and 9000 movies), mainly to make sure there are some (but not too many) empty ratings.

In [35]:
users_frq <- ratings %>% group_by(userId) %>% summarize(count = n()) %>% arrange(desc(count))
my_users <- users_frq$userId[101:115]

Next, we select the 20 movies we want to use:

In [36]:
movies_frq <- ratings %>% group_by(movieId) %>% summarize(count = n()) %>% arrange(desc(count))
my_movies <- movies_frq$movieId[101:120]

Now we make a dataset with only those 15 users and 20 movies:

In [37]:
ratings_red <- ratings %>% filter(userId %in% my_users, movieId %in% my_movies) 
# and check there are 15 users and 20 movies in the reduced dataset
n_users <- length(unique(ratings_red$userId))
n_movies <- length(unique(ratings_red$movieId))
print(paste("number of users is",n_users))
print(paste("number of movies is",n_movies))

[1] "number of users is 15"
[1] "number of movies is 20"


Let's see what the 20 movies are:

In [38]:
movies %>% filter(movieId %in% my_movies) %>% select(title)

title
Taxi Driver (1976)
Waterworld (1995)
Outbreak (1995)
Star Trek: Generations (1994)
Clear and Present Danger (1994)
Casablanca (1942)
"Wizard of Oz, The (1939)"
Apocalypse Now (1979)
Stand by Me (1986)
"Fifth Element, The (1997)"


However, note all the movie titles are still being kept:

In [39]:
levels(ratings_red$title)

[1] "'burbs, The (1989)"                                                                                                                                             
   [2] "'Hellboy': The Seeds of Creation (2004)"                                                                                                                        
   [3] "'Neath the Arizona Skies (1934)"                                                                                                                                
   [4] "'night Mother (1986)"                                                                                                                                           
   [5] "'Round Midnight (1986)"                                                                                                                                         
   [6] "'Salem's Lot (2004)"                                                                                                                                            
   [7] "'Til There Was You (1997)"                                                                                                                                      
   [8] "\"Great Performances\" Cats (1998)"                                                                                                                             
   [9] "$9.99 (2008)"                                                                                                                                                   
  [10] "(500) Days of Summer (2009)"                                                                                                                                    
  [11] "*batteries not included (1987)"                                                                                                                                 
  [12] "...And God Spoke (1993)"                                                                                                                                        
  [13] "...And Justice for All (1979)"                                                                                                                                  
  [14] "[REC] (2007)"                                                                                                                                                   
  [15] "1-900 (06) (1994)"                                                                                                                                              
  [16] "10 (1979)"                                                                                                                                                      
  [17] "10 Attitudes (2001)"                                                                                                                                            
  [18] "10 Cloverfield Lane (2016)"                                                                                                                                     
  [19] "10 Items or Less (2006)"                                                                                                                                        
  [20] "10 Things I Hate About You (1999)"                                                                                                                              
  [21] "10 Years (2011)"                                                                                                                                                
  [22] "10,000 BC (2008)"                                                                                                                                               
  [23] "100 Girls (2000)"                                                                                                                                               
  [24] "100 Rifles (1969)"                                                                                          

This actually isn't what we want, so let's drop the ones we won't use.

In [40]:
ratings_red <- droplevels(ratings_red)
levels(ratings_red$title)

[1] "American Pie (1999)"                                    
 [2] "Apocalypse Now (1979)"                                  
 [3] "Armageddon (1998)"                                      
 [4] "Austin Powers: The Spy Who Shagged Me (1999)"           
 [5] "Beautiful Mind, A (2001)"                               
 [6] "Breakfast Club, The (1985)"                             
 [7] "Casablanca (1942)"                                      
 [8] "Clear and Present Danger (1994)"                        
 [9] "Crouching Tiger, Hidden Dragon (Wo hu cang long) (2000)"
[10] "Fifth Element, The (1997)"                              
[11] "Inception (2010)"                                       
[12] "Kill Bill: Vol. 1 (2003)"                               
[13] "Minority Report (2002)"                                 
[14] "Outbreak (1995)"                                        
[15] "Rain Man (1988)"                                        
[16] "Stand by Me (1986)"                                     
[17] "Star Trek: Generations (1994)"                          
[18] "Taxi Driver (1976)"                                     
[19] "Waterworld (1995)"                                      
[20] "Wizard of Oz, The (1939)"

We now want to reshape the data frame into a 15x20 matrix i.e.from "long" format to "wide" format. We can do this using the `spread()` verb. 

In [41]:
ratings_red %>% spread(key = movieId, value = rating)

userId,timestamp,title,genres,111,208,292,329,349,912,...,1917,1961,1968,2683,2706,3996,4995,5445,6874,79132
149,1436919794,Inception (2010),Action|Crime|Drama|Mystery|Sci-Fi|Thriller|IMAX,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,5.0
149,1436921723,"Breakfast Club, The (1985)",Comedy|Drama,NA,NA,NA,NA,NA,NA,...,NA,NA,4,NA,NA,NA,NA,NA,NA,NA
149,1436923259,"Fifth Element, The (1997)",Action|Adventure|Comedy|Sci-Fi,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
149,1436923311,American Pie (1999),Comedy|Romance,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,3.0,NA,NA,NA,NA,NA
149,1436923357,Stand by Me (1986),Adventure|Drama,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
177,907379242,Armageddon (1998),Action|Romance|Sci-Fi|Thriller,NA,NA,NA,NA,NA,NA,...,3,NA,NA,NA,NA,NA,NA,NA,NA,NA
177,907380001,"Fifth Element, The (1997)",Action|Adventure|Comedy|Sci-Fi,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
177,907380055,Star Trek: Generations (1994),Adventure|Drama|Sci-Fi,NA,NA,NA,4,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
177,907380710,Clear and Present Danger (1994),Action|Crime|Drama|Thriller,NA,NA,NA,NA,4,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
177,907380710,Taxi Driver (1976),Crime|Drama|Thriller,4.0,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


The preceding line *doesn't* work: as you can see we land up with more than one row per user. But it is useful as an illustration of `spread()`. Question: why doesn't it work?

Here's the corrected version:

In [42]:
ratings_red %>% select(userId,title,rating) %>% spread(key = title, value = rating)

userId,American Pie (1999),Apocalypse Now (1979),Armageddon (1998),Austin Powers: The Spy Who Shagged Me (1999),"Beautiful Mind, A (2001)","Breakfast Club, The (1985)",Casablanca (1942),Clear and Present Danger (1994),"Crouching Tiger, Hidden Dragon (Wo hu cang long) (2000)",...,Inception (2010),Kill Bill: Vol. 1 (2003),Minority Report (2002),Outbreak (1995),Rain Man (1988),Stand by Me (1986),Star Trek: Generations (1994),Taxi Driver (1976),Waterworld (1995),"Wizard of Oz, The (1939)"
149,3.0,NA,NA,NA,NA,4.0,NA,NA,NA,...,5.0,NA,NA,NA,NA,4.0,NA,NA,NA,NA
177,NA,NA,3.0,NA,NA,5.0,NA,4.0,NA,...,NA,NA,NA,4,NA,5.0,4.0,4.0,NA,NA
200,1.5,NA,3.0,NA,4.5,NA,NA,NA,NA,...,3.5,NA,4.5,NA,NA,NA,NA,NA,NA,NA
236,NA,5.0,NA,NA,NA,NA,4,NA,3.5,...,NA,NA,NA,NA,NA,NA,NA,4.5,NA,NA
240,NA,NA,NA,NA,3.5,NA,NA,NA,4.0,...,NA,3.0,4.5,NA,NA,NA,NA,5.0,NA,5.0
270,NA,4.0,NA,NA,4.5,NA,NA,NA,NA,...,5.0,5.0,3.5,NA,NA,NA,NA,NA,NA,NA
287,NA,NA,4.0,NA,NA,NA,NA,NA,5.0,...,5.0,NA,5.0,NA,NA,NA,4.5,NA,NA,4.5
295,NA,NA,NA,NA,4.5,NA,4,NA,3.5,...,NA,4.0,4.5,3,4.5,NA,NA,NA,3.5,4.0
303,2.5,NA,NA,NA,4.0,3.5,NA,NA,NA,...,4.0,2.0,NA,NA,4.0,4.5,NA,NA,NA,4.0
408,NA,5.0,4.0,1.0,NA,4.0,NA,NA,NA,...,NA,NA,NA,NA,3.0,4.0,NA,NA,NA,3.0


Finally, since we just want to know who has seen what, we replace all NAs with 0 and all other ratings with 1:

In [43]:
viewed_movies <- ratings_red %>% 
  select(userId,title,rating) %>% 
  complete(userId, title) %>% 
  mutate(seen = ifelse(is.na(rating),0,1)) %>% 
  select(userId,title,seen) %>% 
  spread(key = title, value = seen)

We could have got this more simply with a call to `table()`, which creates a two-way frequency table.

In [44]:
table(ratings_red$userId,ratings_red$title)

     
      American Pie (1999) Apocalypse Now (1979) Armageddon (1998)
  149                   1                     0                 0
  177                   0                     0                 1
  200                   1                     0                 1
  236                   0                     1                 0
  240                   0                     0                 0
  270                   0                     1                 0
  287                   0                     0                 1
  295                   0                     0                 0
  303                   1                     0                 0
  408                   0                     1                 1
  426                   1                     0                 1
  442                   1                     1                 1
  500                   1                     0                 0
  522                   1                     1                 1
  56

Finally, we save our output for use in the next lesson!

In [45]:
save(ratings_red, viewed_movies, file = "output/recommender.RData")

## Combining data frames with *joins*

We'll often need to combine the information contained in two or more tables. To do this, we need various kinds of database *joins*. This section describes the basic join operations that we need to combine data frames. The examples are taken from [Chapter 13](http://r4ds.had.co.nz/relational-data.html) of R4DS, which also contains a lot more general information on relational data.

First, we make some very simple data tables to show how the joins work:

In [46]:
# make some example data
x <- tribble(
  ~key, ~xvalue,
  1, "x1",
  2, "x2",
  3, "x3"
)

y <- tribble(
  ~key, ~yvalue,
  1, "y1",
  2, "y2",
  4, "y3"
)

### Mutating joins: `inner_join`, `left_join`, `right_join`, `full_join`

The first set of joins we look at are called mutating joins. These first match observations in two tables in some way, and then combine variables from the two tables.

There are four types of mutating joins: inner joins, left joins, right joins, and full joins. 

An **inner join** keeps observations that appear in *both* tables. 

In [47]:
inner_join(x,y)

Joining, by = "key"


key,xvalue,yvalue
1,x1,y1
2,x2,y2


In [48]:
inner_join(y,x)

Joining, by = "key"


key,yvalue,xvalue
1,y1,x1
2,y2,x2


The other three joints are all **outer joins**: they keep observations that appear in *at least one* of the tables.

A **left join** keeps all observations in x.

In [49]:
left_join(x,y)

Joining, by = "key"


key,xvalue,yvalue
1,x1,y1
2,x2,y2
3,x3,NA


In [50]:
left_join(y,x)

Joining, by = "key"


key,yvalue,xvalue
1,y1,x1
2,y2,x2
4,y3,NA


A **right join** keeps all observations in y.

In [51]:
# note this is the same as left_join(y,x)
right_join(x,y)

Joining, by = "key"


key,xvalue,yvalue
1,x1,y1
2,x2,y2
4,NA,y3


A **full join** keeps observations in x or y.

In [52]:
full_join(x,y)

Joining, by = "key"


key,xvalue,yvalue
1,x1,y1
2,x2,y2
3,x3,NA
4,NA,y3


We can now re-examine the join we used to add movie titles to the ratings data frame earlier:

In [53]:
# reload the MovieLens data
load("data/movielens-small.RData")
ratings <- as.tibble(ratings)
movies <- as.tibble(movies)

Note that the same *movieId* can appear multiple times in the *ratings* data frame:

In [54]:
print(ratings %>% arrange(movieId)) # note duplicate movieIds

# A tibble: 100,004 x 4
   userId movieId rating  timestamp
    <int>   <int>  <dbl>      <int>
 1      7       1    3.0  851866703
 2      9       1    4.0  938629179
 3     13       1    5.0 1331380058
 4     15       1    2.0  997938310
 5     19       1    3.0  855190091
 6     20       1    3.5 1238729767
 7     23       1    3.0 1148729853
 8     26       1    5.0 1360087980
 9     30       1    4.0  944943070
10     37       1    4.0  981308121
# ... with 99,994 more rows


But each *movieId* only appears once in the *movies* data frame:

In [55]:
print(movies %>% arrange(movieId)) # note unique movieIds

# A tibble: 9,125 x 3
   movieId                              title
     <int>                             <fctr>
 1       1                   Toy Story (1995)
 2       2                     Jumanji (1995)
 3       3            Grumpier Old Men (1995)
 4       4           Waiting to Exhale (1995)
 5       5 Father of the Bride Part II (1995)
 6       6                        Heat (1995)
 7       7                     Sabrina (1995)
 8       8                Tom and Huck (1995)
 9       9                Sudden Death (1995)
10      10                   GoldenEye (1995)
# ... with 9,115 more rows, and 1 more variables: genres <fctr>


In this case a left join by the *movieId* key copies across the movie title information (as well as any other information in the *movies* data frame):

In [56]:
print(left_join(ratings, movies, by = "movieId") %>% select(title,everything()))

# A tibble: 100,004 x 6
                                            title userId movieId rating
                                           <fctr>  <int>   <int>  <dbl>
 1                         Dangerous Minds (1995)      1      31    2.5
 2                                   Dumbo (1941)      1    1029    3.0
 3                                Sleepers (1996)      1    1061    3.0
 4                    Escape from New York (1981)      1    1129    2.0
 5 Cinema Paradiso (Nuovo cinema Paradiso) (1989)      1    1172    4.0
 6                        Deer Hunter, The (1978)      1    1263    2.0
 7                                 Ben-Hur (1959)      1    1287    2.0
 8                                  Gandhi (1982)      1    1293    2.0
 9         Dracula (Bram Stoker's Dracula) (1992)      1    1339    3.5
10                               Cape Fear (1991)      1    1343    2.0
# ... with 99,994 more rows, and 2 more variables: timestamp <int>,
#   genres <fctr>


### Filtering joins: `semi_join`, `anti_join`

The last two joins we look at are **filtering joins**. These match observations in two tables, but do not add variables. There are two types of filtering joins: semi-joins and anti-joins.

A **semi join** keeps all observations in x that appear in y (note variables are from x only),

In [57]:
semi_join(x,y)

Joining, by = "key"


key,xvalue
1,x1
2,x2


In [58]:
semi_join(y,x)

Joining, by = "key"


key,yvalue
1,y1
2,y2


while an **anti join** *drops* all observations in x that appear in y (note variables are from x only).

In [59]:
anti_join(x,y)

Joining, by = "key"


key,xvalue
3,x3


In [60]:
anti_join(y,x)

Joining, by = "key"


key,yvalue
4,y3


## Exercises

Do the exercises in [Chapter 5](http://r4ds.had.co.nz/transform.html) (data transformation using the **dplyr** verbs) and [Chapter 13](http://r4ds.had.co.nz/relational-data.html) (on database joins) of R4DS. There are exercises at the end of each major subsection. Do as many of these exercises as you need to feel comfortable with the material - I suggest doing at least the first two of each set of exercises.